# 計算手順
- ① 状態方程式（Van der Waals式など）を用いて、圧力から液相、気相の体積を計算
- ② 液相と気相のフガシティを計算
- ③ フガシティが等しい時の液相、気相のモル分率を計算

In [1]:
pip install sympy

Note: you may need to restart the kernel to use updated packages.


In [11]:
from scipy.optimize import fsolve
import math
import sympy
import re
import numpy as np

In [18]:
# Variables
T = 460.			#[K] - Temperature
P = 40.*10**(5)			#[Pa] - Pressure
R=8.314			#[J/mol*K] - Universal gas constant
# component 1 = nitrogen
# component 2 = n-Butane
y1 = 0.4974			# Mole percent of nitrogen
y2 = 0.5026			# Mole percent of n-Butane
Tc_nit = 126.2		#[K]
Pc_nit = 34.00*10**(5)			#[Pa]
Tc_but = 425.1			#[K]
Pc_but = 37.96*10**(5)			#[Pa]

# (1). van der Walls equation of state

# The fugacity coefficient of component 1 in a binary mixture following van der Walls equation of state is given by,
# math.log(phi_1) = b_1/(V-b) - math.log(Z-B) -2*(y1*a_11 + y2*a_12)/(R*T*V)
# and for component 2 is given by,
# math.log(phi_2) = b_2/(V-b) - math.log(Z-B) -2*(y1*a_12 + y2*a_22)/(R*T*V)
# Where B = (P*b)/(R*T)

# Calculations
# For componenet 1 (nitrogen)
a_1 = (27*R**(2)*Tc_nit**(2))/(64*Pc_nit)			#[Pa-m**(6)/mol**(2)]
b_1 = (R*Tc_nit)/(8*Pc_nit)			#[m**(3)/mol]

# Similarly for componenet 2 (n-Butane)
a_2 = (27*R**(2)*Tc_but**(2))/(64*Pc_but)		#[Pa-m**(6)/mol**(2)]
b_2 = (R*Tc_but)/(8*Pc_but)			#[m**(3)/mol]

# Here
a_11 = a_1
a_22 = a_2
# For cross coefficient
a_12 = (a_1*a_2)**(1./2)		#[Pa-m**(6)/mol**(2)]

# For the mixture #初期値じゃなくて平衡のときの組成から計算
a = y1**(2)*a_11 + y2**(2)*a_22 + 2*y1*y2*a_12			#[Pa-m**(6)/mol**(2)]
b = y1*b_1 + y2*b_2			#[m**(3)/mol]

# The cubic form of the van der Walls equation of state is given by,
# V**(3) - (b+(R*T)/P)*V**(2) + (a/P)*V - (a*b)/P = 0
# Substituting the value and solving for V, we get
# Solving the cubic equation
def f(V): 
    return V**(3)-(b+(R*T)/P)*V**(2)+(a/P)*V-(a*b)/P
V_1=fsolve(f,-1)
V_2=fsolve(f,0)
V_3=fsolve(f,1)
# The molar volume V = V_3, the other two roots are imaginary
V = V_3			#[m**(3)/mol]

# The comprssibility factor of the mixture is 
Z = (P*V)/(R*T)
# And B can also be calculated as
B = (P*b)/(R*T)

# The fugacity coefficient of component 1 in the mixture is
phi_1 = math.exp(b_1/(V-b) - math.log(Z-B) -2*(y1*a_11 + y2*a_12)/(R*T*V))
# Similarly fugacity coefficient of component 2 in the mixture is 
phi_2 = math.exp(b_2/(V-b) - math.log(Z-B) -2*(y1*a_12 + y2*a_22)/(R*T*V))

# The fugacity coefficient of the mixture is given by,
# math.log(phi) = y1*math.log(phi_1) + y2*math.log(phi_2)
phi = math.exp(y1*math.log(phi_1) + y2*math.log(phi_2))

# Also the fugacity coefficient of the mixture following van der Walls equation of state is given by,
# math.log(phi) = b/(V-b) - math.log(Z-B) -2*a/(R*T*V)
phi_dash = math.exp(b/(V-b) - math.log(Z-B) -2*a/(R*T*V))
# The result is same as obtained above

# Results

###
#print (" 1van der Walls equation of state");
#print ("  The value of fugacity coefficient of component 1 nitrogen) is %f"%(phi_1));
#print ("  The value of fugacity coefficient of component 2 n-butane) is %f"%(phi_2));
#print ("  The value of fugacity coefficient of the mixture is %f"%(phi));
#print ("  Also the fugacity coefficient of the mixture from van der Walls equation of state is %f which is same as above)"%(phi_dash));
###
#print(phi_1,phi_2,phi,phi_dash)

print("\n#####  Results of The fugacity coefficient  #####")
print("       \nφ1: ",phi_1)
print("       \nφ2: ",phi_2)
print(phi)
print(phi_dash)
print(V)


#####  Results of The fugacity coefficient  #####
       
φ1:  1.0574995651175343
       
φ2:  0.801865147851965
0.9201916807315393
0.9201916807315393
[0.0008774]


In [4]:
# when phi_1 = phi_2
# math.exp(b_1/(V-b) - math.log(Z-B) -2*(y1*a_11 + (1-y1)*a_12)/(R*T*V)) 
# = math.exp(b_2/(V-b) - math.log(Z-B) -2*(y1*a_12 + (1-y1)*a_22)/(R*T*V));
# print(b_1/(V-b), b_2/(V-b), math.log(Z-B), a_11, a_12, a_22, R*T*V)
def f(x1):
    return phi_1 * x1 * P
def g(x2):
    return phi_2 * x2 * P
def h(x1, x2):
    return f(x1) - g(x2)
sympy.var('x1y1, x2y2')
#print(1 / (1 + 1.31879976072082))
#print(1.31879976072082 / (1 + 1.31879976072082))
x1y1 = np.array(sympy.solve(phi_1 * x1y1 * P - phi_2 * P, x1y1)) #x1 / y1
x2y2 = np.array(sympy.solve(phi_1 * P - phi_2 * x2y2 * P, x2y2)) #x2 / y2
#print(x1x2, x2x1)
sympy.var('x1, x2')
x = np.zeros(2)
y = np.zeros(2)
x[0] = np.array(sympy.factor(x1 / (x1 + x1 * x2x1)))
x[1] = np.array(sympy.factor(x2 / (x2 + x2 * x1x2)))
print("\n#####  Results of The Mole Percent of each component  #####")
print("       \nx1(nitrogen): ", x[0])
print("       \ny1(nitrogen): ", x[1])

NameError: name 'x2x1' is not defined

In [21]:
from sympy import *
print(V,P,R,T)

[0.0008774] 4000000.0 8.314 460.0


In [30]:
x=Symbol('x')
f = P-(R*T/x)
integrate(f, (x, V, oo))

_CoeffExpValueError: expr not of form a*x**b: 1139.73669985718